In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd

Création des fonctions : 

In [2]:
def A(pN,pM, Xi,m):
    p1 = pN-Xi*(pM-pN)
    pm = pM+Xi*(pM-pN)
    A = [p1]
    for i in range (1,m):
        new = p1 + (i-1)*(pm-p1)/(m-1)
        A.append(new)
    return A

In [3]:
def S(A): 
    S = []
    for i in range (len(A)):
        for j in range (len(A)): 
            S.append([A[i],A[j]])
    return S
    

In [4]:
def pi(s): 
    return (s[0]-1)*np.exp(4*(2-s[0]))/(np.exp(4*(2-s[0]))+np.exp(4*(2-s[1]))-1)

In [5]:
#Cette fonction ne sert pas au final
def env(A,pi):
    env = np.zeros((len(A),len(A)))
    for i in range (len(A)):
        for j in range (len(A)):
            env[i,j] = pi([A[i],A[j]])
    return env       

In [6]:
def initQwithLargeConstantValue(Q): 
    for i in range (len(Q)):
        for j in range(len(Q[0])):
            Q[i][j]=2
    return Q

In [7]:
#Créer une fonction qui trouve le maximum de Q(s,a)
def trouve_max(Q,s_prime):
    sol = Q[0][s_prime]
    arg = 0
    for i in range (1,len(Q)): 
        if sol <= Q[i][s_prime]:
            sol = Q[i][s_prime]
            arg = i
    return sol, arg        

In [8]:
def epsilon(beta,t):
    #return the value of epsilon at the moment t
    return np.exp(-beta*t)

In [9]:
#current_column_index = index of the price set by the players in t-1
def get_next_action(Q,s_t_index,epsilon,m,trouve_max):
    if np.random.random() < 1-epsilon: 
        return trouve_max(Q,s_t_index)[1]
    else: 
        return np.random.randint(m)

In [10]:
def updateQ(Q,S,a_t_index, s_t_index, alpha, delta, pi, s_t1_index):
    Qbis = Q
    for i in range(len(Q)): 
        for j in range(len(Q[0])):
            if i == a_t_index and j == s_t_index: #faire attention à différencier valeurs et indices : ici on parle d'indice
                Qbis[i][j]= (1-alpha)*Q[i][j] + alpha*(pi(S[s_t1_index]) + delta*trouve_max(Q,s_t1_index)[0])
    return Qbis

In [39]:
def find_index(S,s_t): 
    index = -1
    for test in range (len(S)):
        if abs(S[test][0] - s_t[0]) <= 10**(-100) and abs(S[test][1] - s_t[1]) <= 10**(-100) :
            index = test 
    return index

In [12]:
def jeu(pN,pM,Xi,m, A, S, epsilon, beta, alpha, delta, initQ, updateQ, nb_iteration, get_next_action,trouve_max,pi, find_index):
    #Initialisation
    A = A(pN,pM, Xi,m)
    S = S(A)
    Q1 = np.zeros((len(A),len(S))) #Q matrice j1
    Q1 = initQ(Q1)
    Q2 = np.zeros((len(A),len(S))) #Q matrice j2
    Q2 = initQ(Q2)
    t = 0
    s_t_index = np.random.randint(m*m)
    #Phase itérative
    while t < nb_iteration:
        a_t_index1 = get_next_action(Q1,s_t_index,epsilon(beta,t),m,trouve_max)
        a_t_index2 = get_next_action(Q2,s_t_index,epsilon(beta,t),m,trouve_max)
        s_t1 = [A[a_t_index1],A[a_t_index2]]
        s_t1_index = find_index(S,s_t1)
        Q1 = updateQ(Q1,S, a_t_index1, s_t_index, alpha, delta, pi, s_t1_index)
        Q2 = updateQ(Q2,S,a_t_index2, s_t_index, alpha, delta, pi, s_t1_index)
        t = t+1
        s_t_index = s_t1_index
    return Q, S[s_t1_index]
    

Test des différentes fonctions

In [13]:
A_test = A(1.78,2,0.1,15)
print(A_test)

[1.758, 1.758, 1.776857142857143, 1.7957142857142856, 1.8145714285714285, 1.8334285714285714, 1.8522857142857143, 1.871142857142857, 1.89, 1.9088571428571428, 1.9277142857142855, 1.9465714285714284, 1.9654285714285713, 1.9842857142857142, 2.003142857142857]


In [14]:
S_test = S(A_test)
print(S_test)

[[1.758, 1.758], [1.758, 1.758], [1.758, 1.776857142857143], [1.758, 1.7957142857142856], [1.758, 1.8145714285714285], [1.758, 1.8334285714285714], [1.758, 1.8522857142857143], [1.758, 1.871142857142857], [1.758, 1.89], [1.758, 1.9088571428571428], [1.758, 1.9277142857142855], [1.758, 1.9465714285714284], [1.758, 1.9654285714285713], [1.758, 1.9842857142857142], [1.758, 2.003142857142857], [1.758, 1.758], [1.758, 1.758], [1.758, 1.776857142857143], [1.758, 1.7957142857142856], [1.758, 1.8145714285714285], [1.758, 1.8334285714285714], [1.758, 1.8522857142857143], [1.758, 1.871142857142857], [1.758, 1.89], [1.758, 1.9088571428571428], [1.758, 1.9277142857142855], [1.758, 1.9465714285714284], [1.758, 1.9654285714285713], [1.758, 1.9842857142857142], [1.758, 2.003142857142857], [1.776857142857143, 1.758], [1.776857142857143, 1.758], [1.776857142857143, 1.776857142857143], [1.776857142857143, 1.7957142857142856], [1.776857142857143, 1.8145714285714285], [1.776857142857143, 1.833428571428571

In [15]:
print([S_test[90],pi(S_test[90])])

[[1.8522857142857143, 1.758], 0.4475678764122718]


In [16]:
#Fonction qui ne sert pas au final mais testons la tout de même
env = env(A_test,pi)
print(env)

[[0.46785559 0.46785559 0.48982103 0.51211767 0.53468835 0.55747289
  0.58040877 0.60343175 0.62647662 0.64947792 0.67237067 0.69509113
  0.71757748 0.73977048 0.76161406]
 [0.46785559 0.46785559 0.48982103 0.51211767 0.53468835 0.55747289
  0.58040877 0.60343175 0.62647662 0.64947792 0.67237067 0.69509113
  0.71757748 0.73977048 0.76161406]
 [0.46553375 0.46553375 0.48846686 0.51184964 0.53562709 0.55974008
  0.58412587 0.60871871 0.63345058 0.65825186 0.68305217 0.70778112
  0.73236915 0.75674826 0.7808528 ]
 [0.46231851 0.46231851 0.48618346 0.51062704 0.53559861 0.56104225
  0.58689721 0.61309834 0.63957679 0.66626064 0.69307565 0.71994615
  0.74679581 0.77354854 0.80012935]
 [0.45823267 0.45823267 0.48298553 0.50845584 0.53459977 0.56136718
  0.58870191 0.61654211 0.64482069 0.67346592 0.70240208 0.73155024
  0.76082911 0.79015592 0.81944735]
 [0.45330456 0.45330456 0.47889355 0.5053478  0.53263293 0.56070756
  0.58952323 0.61902458 0.64914962 0.67983008 0.71099195 0.74255611
  0.

In [17]:
Q = np.zeros((len(A_test),len(S_test)))
Q = initQwithLargeConstantValue(Q)
print(Q)

[[2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 ...
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]]


In [18]:
Q = [[1,2,3],[4,5,6],[7,8,9]]
trouve_max(Q,0)

(7, 2)

In [19]:
for t in range (10): 
    print(epsilon(10**(-5),t))

1.0
0.9999900000499998
0.9999800001999987
0.9999700004499955
0.9999600007999894
0.9999500012499791
0.999940001799964
0.9999300024499428
0.9999200031999147
0.9999100040498785


In [20]:
Q = [[1,2,3],[4,5,6],[7,8,9]]
print(get_next_action(Q,1,0.5,15,trouve_max))

6


In [21]:
Q = np.zeros((len(A_test),len(S_test)))
Q = initQwithLargeConstantValue(Q)
print(updateQ(Q,S_test,0,0, 0.1, 0.95, pi, np.random.randint(225))) #Ca change bien quand on change s_t1_index

[[2.08760473 2.         2.         ... 2.         2.         2.        ]
 [2.         2.         2.         ... 2.         2.         2.        ]
 [2.         2.         2.         ... 2.         2.         2.        ]
 ...
 [2.         2.         2.         ... 2.         2.         2.        ]
 [2.         2.         2.         ... 2.         2.         2.        ]
 [2.         2.         2.         ... 2.         2.         2.        ]]


In [41]:
#for i in range (len(S)):
    #print(i,find_index(S,S[i])) 
for i in range (len(S_test)):
    if i != find_index(S_test,S_test[i]):
        print(i,find_index(S_test,S_test[i])) 
        print(S_test[i],S_test[find_index(S_test,S_test[i])])#ptit pb arrondis (pour S[90] ça vaut 91, pour S[0], 16 !!) à résoudre 

0 16
[1.758, 1.758] [1.758, 1.758]
1 16
[1.758, 1.758] [1.758, 1.758]
2 17
[1.758, 1.776857142857143] [1.758, 1.776857142857143]
3 18
[1.758, 1.7957142857142856] [1.758, 1.7957142857142856]
4 19
[1.758, 1.8145714285714285] [1.758, 1.8145714285714285]
5 20
[1.758, 1.8334285714285714] [1.758, 1.8334285714285714]
6 21
[1.758, 1.8522857142857143] [1.758, 1.8522857142857143]
7 22
[1.758, 1.871142857142857] [1.758, 1.871142857142857]
8 23
[1.758, 1.89] [1.758, 1.89]
9 24
[1.758, 1.9088571428571428] [1.758, 1.9088571428571428]
10 25
[1.758, 1.9277142857142855] [1.758, 1.9277142857142855]
11 26
[1.758, 1.9465714285714284] [1.758, 1.9465714285714284]
12 27
[1.758, 1.9654285714285713] [1.758, 1.9654285714285713]
13 28
[1.758, 1.9842857142857142] [1.758, 1.9842857142857142]
14 29
[1.758, 2.003142857142857] [1.758, 2.003142857142857]
15 16
[1.758, 1.758] [1.758, 1.758]
30 31
[1.776857142857143, 1.758] [1.776857142857143, 1.758]
45 46
[1.7957142857142856, 1.758] [1.7957142857142856, 1.758]
60 61
[1

In [36]:
jeu(pN=1.78,pM=2,Xi=0.1,m=15, A=A, S=S, epsilon = epsilon, beta=10**(-5), alpha = 0.1, delta = 0.95, initQ=initQwithLargeConstantValue, updateQ=updateQ, nb_iteration=10000, get_next_action = get_next_action,trouve_max=trouve_max,pi=pi, find_index=find_index)

(array([[2.08760473, 2.        , 2.        , ..., 2.        , 2.        ,
         2.        ],
        [2.        , 2.        , 2.        , ..., 2.        , 2.        ,
         2.        ],
        [2.        , 2.        , 2.        , ..., 2.        , 2.        ,
         2.        ],
        ...,
        [2.        , 2.        , 2.        , ..., 2.        , 2.        ,
         2.        ],
        [2.        , 2.        , 2.        , ..., 2.        , 2.        ,
         2.        ],
        [2.        , 2.        , 2.        , ..., 2.        , 2.        ,
         2.        ]]),
 [1.871142857142857, 1.9088571428571428])